In [1]:
# from helper_functions.tourney_sim import simulate_tournament
#in v02 im just going to test 8 combo
#v03 calculate expected value format
#v04 do the same for generalized to n=4 games
import os
import numpy as np
import pickle
from tqdm import tqdm
from itertools import combinations, product
import pandas as pd
pd.set_option('display.max_columns', 90)

In [2]:
def implied_prob(line):
    if line < 0:
        ip = (-1*line)/(-1*line+100)
    else:
        ip=100/(line+100)
    return ip

In [3]:
def reverse_prob(p):
    if p <= .5:
        line = (100/p)-100
    else:
        line = (100*p)/(p-1)
    return line


In [4]:
def payout(line, bet):
    winnings=0
    if line < 0:
        winnings = bet/(-line/100)
    else:
        winnings = bet * (line/100)
    return (bet + winnings)

In [5]:
def combine_lines(lines=[]):
    probs=[]
    for l in lines:
        probs.append(implied_prob(l))
    total_prob=1
    for p in probs:
        total_prob*=p
    return reverse_prob(total_prob)


In [6]:
def find_ones_indexes(binary_tuple):
    indexes = []
    for index, bit in enumerate(binary_tuple):
        if bit == 1:
            indexes.append(index+1)
    return indexes

In [7]:
lines1 = {'Marqette':-285,
         'Purdue':-230,
         'Houston':-192,
         'Tennessee':-148}

In [8]:
lines2 = {'NC State':230,
         'Gonzaga':190,
         'Duke':160,
         'Creighton':124}

In [9]:
probs1={'Marqette':.832,
       'Purdue':.566,
       'Houston':.416,
       'Tennessee':.481}

In [10]:
probs2={'NC State':.168,
       'Gonzaga':.434,
       'Duke':.584,
       'Creighton':.519}

In [11]:
lines_comb = dict(zip(list(lines1.keys()) + list(lines2.keys()),
                      list(lines1.values())+list(lines2.values())))

probs_comb = dict(zip(list(probs1.keys()) + list(probs2.keys()),
                      list(probs1.values())+list(probs2.values())))

In [12]:
lines_df = pd.DataFrame(zip(lines1,lines2))

In [13]:

n=4
N=2**n

# Generate all possible combinations of three bytes
combos = product(range(N), repeat=3)

# Convert each combination into the desired format
data = tqdm([tuple(tuple(int(bit) for bit in bin(byte)[2:].zfill(n)) for byte in combo) for combo in combos])

# Create DataFrame
df = pd.DataFrame(data, columns=['Byte1', 'Byte2', 'Byte3'])

100%|██████████| 4096/4096 [00:00<00:00, 4094344.42it/s]


In [15]:
df.columns=['Picks','Winners','RR Choice']

In [16]:
def map_byte_to_teams(b):
    return str(list(map(lambda x,y: lines_df.loc[x][y], range(n), b)))

vec_map = np.vectorize(map_byte_to_teams)

In [17]:
kreb = df['Picks'].unique()

In [18]:
len(kreb)

16

In [19]:
all_teams = vec_map(kreb)

In [20]:
all_probs = []
for team_combo in all_teams:
    starting_p = 1
    winner_list = team_combo.strip("[]''").split("', '")
    for w in winner_list:
        starting_p *= probs_comb[w]
    all_probs.append(starting_p)

In [21]:
# all_teams

In [22]:
#make success dict
outer_successes = dict()
for pick_combo in tqdm(all_teams):
    pick_list = pick_combo.strip("[]''").split("', '")
    inner_dict = dict()
    for winner_combo in all_teams:
        winner_list = winner_combo.strip("[]''").split("', '")
        success_list = [winner_list[x] == pick_list[x] for x in range(len(pick_list))]
        inner_dict[winner_combo] = success_list
    outer_successes[pick_combo] = inner_dict


100%|██████████| 16/16 [00:00<00:00, 13986.84it/s]


In [23]:
probs_dict = dict(zip(kreb, all_probs))

In [24]:
choices_dict = dict(zip(kreb, all_teams))

In [25]:
df['Pick_List'] = [choices_dict[x] for x in df['Picks']]

In [26]:
df['Winner_List'] = [choices_dict[x] for x in df['Winners']]

In [27]:
df['Outcome_Prob'] = [probs_dict[x] for x in df['Winners']]

In [28]:
df['Success_List'] = list(map(lambda x,y: outer_successes[x][y], df['Pick_List'], df['Winner_List']))

In [29]:
df.head()

,Picks,Winners,RR Choice,Pick_List,Winner_List,Outcome_Prob,Success_List
0,"(0, 0, 0, 0)","(0, 0, 0, 0)","(0, 0, 0, 0)","['Marqette', 'Purdue', 'Houston', 'Tennessee']","['Marqette', 'Purdue', 'Houston', 'Tennessee']",0.094228,"[True, True, True, True]"
1,"(0, 0, 0, 0)","(0, 0, 0, 0)","(0, 0, 0, 1)","['Marqette', 'Purdue', 'Houston', 'Tennessee']","['Marqette', 'Purdue', 'Houston', 'Tennessee']",0.094228,"[True, True, True, True]"
2,"(0, 0, 0, 0)","(0, 0, 0, 0)","(0, 0, 1, 0)","['Marqette', 'Purdue', 'Houston', 'Tennessee']","['Marqette', 'Purdue', 'Houston', 'Tennessee']",0.094228,"[True, True, True, True]"
3,"(0, 0, 0, 0)","(0, 0, 0, 0)","(0, 0, 1, 1)","['Marqette', 'Purdue', 'Houston', 'Tennessee']","['Marqette', 'Purdue', 'Houston', 'Tennessee']",0.094228,"[True, True, True, True]"
4,"(0, 0, 0, 0)","(0, 0, 0, 0)","(0, 1, 0, 0)","['Marqette', 'Purdue', 'Houston', 'Tennessee']","['Marqette', 'Purdue', 'Houston', 'Tennessee']",0.094228,"[True, True, True, True]"


In [31]:
working_df = df[['Pick_List','Winner_List','Outcome_Prob','RR Choice','Success_List']].copy()

In [33]:
del df

In [34]:
def calculate_rr_winning(picks, success, rr_combo):
    """
    current_rrs = find_ones_indexes(test_rr)
    """
    current_rrs = find_ones_indexes(rr_combo)
    rr_moneys=[]
    pick_list = picks.strip("[]''").split("', '")
    success_dict = dict(zip(pick_list,success))
    



    for i in current_rrs:
        current_combinations =list(combinations(pick_list,i))
        for combo in current_combinations:
            current_payout = 0
            cost=0
            line = combine_lines([lines_comb[x] for x in combo])
            cost += 0.1
            win = True
            for choice in combo:
                if success_dict[choice] == 0:
                    win=False
            if win:
                current_payout += payout(line,0.1)
        rr_moneys.append(current_payout-cost)
    return np.sum(rr_moneys)

In [35]:
vec_rr = np.vectorize(calculate_rr_winning)

In [36]:
%%time
working_df['Winnings']=vec_rr(working_df['Pick_List'], working_df['Success_List'], working_df['RR Choice'])

Wall time: 63 ms


In [37]:
working_df

,Pick_List,Winner_List,Outcome_Prob,RR Choice,Success_List,Winnings
0,"['Marqette', 'Purdue', 'Houston', 'Tennessee']","['Marqette', 'Purdue', 'Houston', 'Tennessee']",0.094228,"(0, 0, 0, 0)","[True, True, True, True]",0.000000
1,"['Marqette', 'Purdue', 'Houston', 'Tennessee']","['Marqette', 'Purdue', 'Houston', 'Tennessee']",0.094228,"(0, 0, 0, 1)","[True, True, True, True]",0.393939
2,"['Marqette', 'Purdue', 'Houston', 'Tennessee']","['Marqette', 'Purdue', 'Houston', 'Tennessee']",0.094228,"(0, 0, 1, 0)","[True, True, True, True]",0.265643
3,"['Marqette', 'Purdue', 'Houston', 'Tennessee']","['Marqette', 'Purdue', 'Houston', 'Tennessee']",0.094228,"(0, 0, 1, 1)","[True, True, True, True]",0.659583
4,"['Marqette', 'Purdue', 'Houston', 'Tennessee']","['Marqette', 'Purdue', 'Houston', 'Tennessee']",0.094228,"(0, 1, 0, 0)","[True, True, True, True]",0.154842
...,...,...,...,...,...,...
4091,"['NC State', 'Gonzaga', 'Duke', 'Creighton']","['NC State', 'Gonzaga', 'Duke', 'Creighton']",0.022099,"(1, 0, 1, 1)","[True, True, True, True]",7.186528
4092,"['NC State', 'Gonzaga', 'Duke', 'Creighton']","['NC State', 'Gonzaga', 'Duke', 'Creighton']",0.022099,"(1, 1, 0, 0)","[True, True, True, True]",0.606400
4093,"['NC State', 'Gonzaga', 'Duke', 'Creighton']","['NC State', 'Gonzaga', 'Duke', 'Creighton']",0.022099,"(1, 1, 0, 1)","[True, True, True, True]",6.079968
4094,"['NC State', 'Gonzaga', 'Duke', 'Creighton']","['NC State', 'Gonzaga', 'Duke', 'Creighton']",0.022099,"(1, 1, 1, 0)","[True, True, True, True]",2.195360


In [38]:
working_df['EV'] = working_df['Outcome_Prob'] * working_df['Winnings']

In [39]:
grps = working_df.groupby(['Pick_List','RR Choice'])[['Outcome_Prob','EV']].sum().reset_index()

In [40]:
grps.sort_values(by='EV').reset_index(drop=True)#.loc[65535]['Pick_List']

,Pick_List,RR Choice,Outcome_Prob,EV
0,"['NC State', 'Purdue', 'Houston', 'Tennessee']","(1, 1, 1, 1)",1.0,-0.204038
1,"['NC State', 'Purdue', 'Houston', 'Tennessee']","(0, 1, 1, 1)",1.0,-0.184638
2,"['Marqette', 'Purdue', 'Houston', 'Tennessee']","(1, 1, 1, 1)",1.0,-0.180454
3,"['NC State', 'Gonzaga', 'Houston', 'Tennessee']","(1, 1, 1, 1)",1.0,-0.168646
4,"['Marqette', 'Purdue', 'Houston', 'Tennessee']","(0, 1, 1, 1)",1.0,-0.161054
...,...,...,...,...
251,"['Marqette', 'Gonzaga', 'Duke', 'Creighton']","(1, 1, 0, 1)",1.0,0.242485
252,"['Marqette', 'Gonzaga', 'Duke', 'Creighton']","(0, 0, 1, 1)",1.0,0.271878
253,"['Marqette', 'Gonzaga', 'Duke', 'Creighton']","(1, 0, 1, 1)",1.0,0.288134
254,"['Marqette', 'Gonzaga', 'Duke', 'Creighton']","(0, 1, 1, 1)",1.0,0.348401


In [144]:
working_df[(working_df['Pick_List']=="['Clemson', 'San Diego St.', 'North Carolina', 'Iowa St.', 'Marqette', 'Gonzaga', 'Duke', 'Creighton']")
           &(working_df['RR Choice']==(1,1,1,1,1,1,1,1))].sort_values(by='Success_List')

,Pick_List,Winner_List,Outcome_Prob,RR Choice,Success_List,Winnings,EV
13056255,"['Clemson', 'San Diego St.', 'North Carolina',...","['Arizona', 'UConn', 'Alabama', 'Illinois', 'N...",0.000430,"(1, 1, 1, 1, 1, 1, 1, 1)","[False, False, False, False, False, False, Fal...",-0.800000,-0.000344
13056511,"['Clemson', 'San Diego St.', 'North Carolina',...","['Arizona', 'UConn', 'Alabama', 'Illinois', 'N...",0.000464,"(1, 1, 1, 1, 1, 1, 1, 1)","[False, False, False, False, False, False, Fal...",-0.576000,-0.000267
13056767,"['Clemson', 'San Diego St.', 'North Carolina',...","['Arizona', 'UConn', 'Alabama', 'Illinois', 'N...",0.000603,"(1, 1, 1, 1, 1, 1, 1, 1)","[False, False, False, False, False, False, Tru...",-0.800000,-0.000483
13057023,"['Clemson', 'San Diego St.', 'North Carolina',...","['Arizona', 'UConn', 'Alabama', 'Illinois', 'N...",0.000651,"(1, 1, 1, 1, 1, 1, 1, 1)","[False, False, False, False, False, False, Tru...",0.006400,0.000004
13057279,"['Clemson', 'San Diego St.', 'North Carolina',...","['Arizona', 'UConn', 'Alabama', 'Illinois', 'N...",0.000329,"(1, 1, 1, 1, 1, 1, 1, 1)","[False, False, False, False, False, True, Fals...",-0.800000,-0.000264
...,...,...,...,...,...,...,...
13091839,"['Clemson', 'San Diego St.', 'North Carolina',...","['Clemson', 'San Diego St.', 'North Carolina',...",0.007929,"(1, 1, 1, 1, 1, 1, 1, 1)","[True, True, True, True, True, False, True, True]",0.006400,0.000051
13092095,"['Clemson', 'San Diego St.', 'North Carolina',...","['Clemson', 'San Diego St.', 'North Carolina',...",0.004014,"(1, 1, 1, 1, 1, 1, 1, 1)","[True, True, True, True, True, True, False, Fa...",-0.800000,-0.003211
13092351,"['Clemson', 'San Diego St.', 'North Carolina',...","['Clemson', 'San Diego St.', 'North Carolina',...",0.004331,"(1, 1, 1, 1, 1, 1, 1, 1)","[True, True, True, True, True, True, False, True]",-0.576000,-0.002494
13092607,"['Clemson', 'San Diego St.', 'North Carolina',...","['Clemson', 'San Diego St.', 'North Carolina',...",0.005634,"(1, 1, 1, 1, 1, 1, 1, 1)","[True, True, True, True, True, True, True, False]",-0.800000,-0.004507


In [ ]:
vec_rr(small_df['Pick_List'], small_df['Success_List'], small_df['RR Choice'])

In [73]:
small_df

,Pick_List,Winner_List,Outcome_Prob,RR Choice,Success_List,Winnings
0,"['Arizona', 'UConn', 'North Carolina', 'Iowa S...","['Arizona', 'UConn', 'North Carolina', 'Iowa S...",0.024962,"(0, 0, 0, 0, 0, 0, 0, 0)","[True, True, True, True, True, True, True, True]",0.000000
1,"['Arizona', 'UConn', 'North Carolina', 'Iowa S...","['Arizona', 'UConn', 'North Carolina', 'Iowa S...",0.024962,"(0, 0, 0, 0, 0, 0, 0, 1)","[True, True, True, True, True, True, True, True]",1.912762
2,"['Arizona', 'UConn', 'North Carolina', 'Iowa S...","['Arizona', 'UConn', 'North Carolina', 'Iowa S...",0.024962,"(0, 0, 0, 0, 0, 0, 1, 0)","[True, True, True, True, True, True, True, True]",10.371843
3,"['Arizona', 'UConn', 'North Carolina', 'Iowa S...","['Arizona', 'UConn', 'North Carolina', 'Iowa S...",0.024962,"(0, 0, 0, 0, 0, 0, 1, 1)","[True, True, True, True, True, True, True, True]",22.656449
4,"['Arizona', 'UConn', 'North Carolina', 'Iowa S...","['Arizona', 'UConn', 'North Carolina', 'Iowa S...",0.024962,"(0, 0, 0, 0, 0, 1, 0, 0)","[True, True, True, True, True, True, True, True]",24.252095
...,...,...,...,...,...,...
9996,"['Arizona', 'UConn', 'North Carolina', 'Iowa S...","['Arizona', 'UConn', 'Alabama', 'Iowa St.', 'M...",0.006451,"(0, 0, 0, 0, 1, 1, 0, 0)","[True, True, False, True, True, False, False, ...",-14.000000
9997,"['Arizona', 'UConn', 'North Carolina', 'Iowa S...","['Arizona', 'UConn', 'Alabama', 'Iowa St.', 'M...",0.006451,"(0, 0, 0, 0, 1, 1, 0, 1)","[True, True, False, True, True, False, False, ...",-22.500000
9998,"['Arizona', 'UConn', 'North Carolina', 'Iowa S...","['Arizona', 'UConn', 'Alabama', 'Iowa St.', 'M...",0.006451,"(0, 0, 0, 0, 1, 1, 1, 0)","[True, True, False, True, True, False, False, ...",-23.200000
9999,"['Arizona', 'UConn', 'North Carolina', 'Iowa S...","['Arizona', 'UConn', 'Alabama', 'Iowa St.', 'M...",0.006451,"(0, 0, 0, 0, 1, 1, 1, 1)","[True, True, False, True, True, False, False, ...",-32.500000


In [107]:
i=3

In [108]:
p=working_df.iloc[i]['Pick_List']
s=working_df.iloc[i]['Success_List']
r=working_df.iloc[i]['RR Choice']

In [109]:
calculate_rr_winning(p,s,r)

[7, 8]


3.3346067637018413

In [37]:
test_rr = (1,1,0,0,0,0,0,0,)

In [41]:
test_picks = working_df['Pick_List'][0]

In [42]:
test_picks

"['Arizona', 'UConn', 'North Carolina', 'Iowa St.', 'Marqette', 'Purdue', 'Houston', 'Tennessee']"

In [49]:
current_rrs

[1, 2]

In [57]:
test_rr

(1, 1, 0, 0, 0, 0, 0, 0)

In [58]:
current_rrs = find_ones_indexes(test_rr)
cost = 0
current_payout = 0
rr_moneys=[]
pick_list = test_picks.strip("[]''").split("', '")
success_dict = dict(zip(pick_list,(1,1,1,1,1,1,1,1)))
for i in current_rrs:
    current_combinations =list(combinations(pick_list,i))
    for combo in current_combinations:
        line = combine_lines([lines_comb[x] for x in combo])
        cost += 0.1
        win = True
        for choice in combo:
            if success_dict[choice] == 0:
                win=False
        if win:
            current_payout += payout(line,0.1)
    rr_moneys.append(current_payout-cost)
print(np.sum(rr_moneys))

3.979499047886855


In [56]:
rr_moneys

[0.3754472092046456, 3.6040518386822096]